In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import numpy as np
import cv2
import os
from skimage.morphology import convex_hull_image, area_closing

In [ ]:
#  load all images from a directory
def load_images_from_folder(folder):
    images = []
    filenames = []
    for filename in tqdm(os.listdir(folder)):
        try:
            img = Image.open(os.path.join(folder,filename))
            if img is not None:
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                images.append(img)
                filenames.append(filename)
        except Exception as e:
            print(e)
    return images, filenames

In [ ]:
# load all jsons from a directory
def load_jsons_from_folder(folder):
    jsons = []
    filenames = []
    for filename in tqdm(os.listdir(folder)):
        try:
            json = pd.read_json(os.path.join(folder,filename))
            if json is not None:
                jsons.append(json)
                filenames.append(filename)
        except Exception as e:
            print(e)
    return jsons, filenames

In [ ]:
human_parsing = load_images_from_folder('Task/dataset/human_parsing')
image = load_images_from_folder('Task/dataset/image')
pose_img = load_images_from_folder('Task/dataset/pose_img')
pose_json = load_jsons_from_folder('Task/dataset/pose_json')

In [ ]:
df = pd.DataFrame()
df['human_parsing'] = human_parsing[0]
df['image'] = image[0]
df['pose_img'] = pose_img[0]
df['pose_json'] = pose_json[0]

In [ ]:
# assert all(human_parsing[1] == image[1] == pose_img[1])
df_names = pd.DataFrame()
df_names['human_parsing'] = human_parsing[1]
df_names['image'] = image[1]
df_names['pose_img'] = pose_img[1]
df_names['pose_json'] = pose_json[1]

In [ ]:
def show_images_df(df, index):
    fig, axs = plt.subplots(1, 3, figsize=(20, 20))
    axs[0].imshow(df['human_parsing'][index])
    axs[1].imshow(df['image'][index])
    axs[2].imshow(df['pose_img'][index])

In [ ]:
show_images_df(df, 0)

In [ ]:
df['pose_keypoints'] = df.pose_json.apply(lambda x: x.people[0].get('pose_keypoints_2d'))

In [ ]:
# draw circles for each dot in poses and add annotation with number of big size
def draw_pose(img, poses):
    draw = ImageDraw.Draw(img)
    for i in range(0, len(poses), 3):
        draw.ellipse((poses[i]-5, poses[i+1]-5, poses[i]+5, poses[i+1]+5), fill=(255, 0, 0, 255))
        draw.text((poses[i], poses[i+1]), str(i//3), fill=(255, 0, 0, 0), font= ImageFont.truetype("arial.ttf", 30))
    return img
draw_pose(df.image[0], df.pose_keypoints[0])

In [ ]:
short_colors = {(64, 0, 128), (192, 0, 0)}
legs_colors = {(128,64,0), (0,64,0)}
skirt_colors = short_colors.union(legs_colors)
skirt_colors

In [ ]:
shirt_colors = {(128,0,128)}
hands_colors = {(192,128,128),(64,128,128)}
body_colors = shirt_colors.union(hands_colors)

In [ ]:
def get_color_mask(img, colors):
    img = np.array(img)
    mask = np.zeros(img.shape[:2], dtype=np.uint8)
    for color in colors:
        mask += np.all(img == color, axis=-1).astype(np.uint8)
    return mask

In [ ]:
mask_legs_and_shorts = [get_color_mask(a, skirt_colors) for a in tqdm(human_parsing[0])]

In [ ]:
mask_upper_body = [get_color_mask(a, body_colors) for a in tqdm(human_parsing[0])]

In [ ]:
mask_shirts = [get_color_mask(a, shirt_colors) for a in tqdm(human_parsing[0])]

In [98]:
mask_shirts_closed = [area_closing(a, 300) for a in tqdm(mask_shirts)]

In [ ]:
# convex_shirts = [convex_hull_image(a) for a in tqdm(mask_shirts)]
convex_skirts = [convex_hull_image(a) for a in tqdm(mask_legs_and_shorts)]

In [ ]:
df_names.loc[df_names.image.str.contains('48366')]

In [ ]:
imshow(convex_shirts[235])

In [ ]:
imshow(area_closing(mask_shirts[235], area_threshold=1000))

In [ ]:
skirt = convex_skirts[235]
shirt = area_closing(mask_shirts[235], 1000)
imshow(skirt^skirt*shirt)

In [ ]:
# apply skirt mask as gray color
def apply_mask(img, mask):
    img = np.array(img)
    img[mask] = (128, 128, 128)
    return Image.fromarray(img)

In [ ]:
processed_images = [apply_mask(a, b) for a, b in tqdm(zip(df.image, convex_skirts))]

In [ ]:
df['processed_images'] = processed_images

In [ ]:
# save processed_images to /preprocessed
for i, img in tqdm(enumerate(processed_images)):
    img.save('C:/code/python/test_task_big/preprocessed/' + df_names.image[i])

In [ ]:
# todo:
# delete regions of skirt mask that are inside 